<a href="https://colab.research.google.com/github/Xeesto/UEP/blob/main/BD_Wyspa_Skarbow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Ustaw wersję jako parametr
SPARK_VERSION="3.5.5"

# Instalacja OpenJDK 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Pobranie Apache Spark z określoną wersją
!wget -q http://www.apache.org/dist/spark/spark-$SPARK_VERSION/spark-$SPARK_VERSION-bin-hadoop3.tgz

# Rozpakowanie archiwum Spark
!tar xf spark-$SPARK_VERSION-bin-hadoop3.tgz

# Instalacja findspark i pyspark
!pip install -q findspark==1.3.0
!pip install -q pyspark==$SPARK_VERSION

# Ustalamy zmienne środowiskowe.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-{SPARK_VERSION}-bin-hadoop3"

In [2]:
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext

findspark.init(f"spark-{SPARK_VERSION}-bin-hadoop3")
sc = pyspark.SparkContext('local[*]')
spark = SparkSession.builder.appName('abc').getOrCreate()

In [3]:
!free -mh # Komenda na sprawdzenie dostępnej pamięci RAM

               total        used        free      shared  buff/cache   available
Mem:            12Gi       1.2Gi       3.8Gi       1.0Mi       7.7Gi        11Gi
Swap:             0B          0B          0B


In [4]:
!head -5 baby.csv

Year,First Name,County,Sex,Count
2013,DAVID,KINGS,M,272
2013,JAYDEN,KINGS,M,268
2013,JAYDEN,QUEENS,M,219
2013,MOSHE,KINGS,M,219


In [ ]:
# from google.colab import files
# baby_names = files_upload()


In [7]:
baby_names = sc.textFile('baby.csv')

baby_names_no_header = baby_names.filter(lambda x: x!='Year,First Name,County,Sex,Count')

In [8]:
baby_names_no_header.take(10)

['2013,DAVID,KINGS,M,272',
 '2013,JAYDEN,KINGS,M,268',
 '2013,JAYDEN,QUEENS,M,219',
 '2013,MOSHE,KINGS,M,219',
 '2013,ETHAN,QUEENS,M,216',
 '2013,SOPHIA,QUEENS,F,204',
 '2013,DANIEL,QUEENS,M,203',
 '2013,JACOB,KINGS,M,202',
 '2013,ESTHER,KINGS,F,196',
 '2013,ETHAN,KINGS,M,196']

In [10]:
rows = baby_names_no_header.map(lambda x: x.split(','))

In [12]:
(rows
 .map(lambda x: (x[1], x[2]))
 .groupByKey()
 .map(lambda x: {x[0]: list(x[1])})
 .collect()
)

[{'DAVID': ['KINGS']},
 {'ETHAN': ['QUEENS', 'KINGS']},
 {'JACOB': ['KINGS', 'QUEENS']},
 {'ESTHER': ['KINGS']},
 {'ISABELLA': ['QUEENS']},
 {'LEAH': ['KINGS']},
 {'NOAH': ['KINGS']},
 {'JOSEPH': ['KINGS']},
 {'MATTHEW': ['QUEENS']},
 {'JAYDEN': ['KINGS', 'QUEENS']},
 {'MOSHE': ['KINGS']},
 {'SOPHIA': ['QUEENS']},
 {'DANIEL': ['QUEENS', 'KINGS']},
 {'AIDEN': ['KINGS']}]

In [13]:
# Wyświetlanie
rows.take(20)

[['2013', 'DAVID', 'KINGS', 'M', '272'],
 ['2013', 'JAYDEN', 'KINGS', 'M', '268'],
 ['2013', 'JAYDEN', 'QUEENS', 'M', '219'],
 ['2013', 'MOSHE', 'KINGS', 'M', '219'],
 ['2013', 'ETHAN', 'QUEENS', 'M', '216'],
 ['2013', 'SOPHIA', 'QUEENS', 'F', '204'],
 ['2013', 'DANIEL', 'QUEENS', 'M', '203'],
 ['2013', 'JACOB', 'KINGS', 'M', '202'],
 ['2013', 'ESTHER', 'KINGS', 'F', '196'],
 ['2013', 'ETHAN', 'KINGS', 'M', '196'],
 ['2013', 'ISABELLA', 'QUEENS', 'F', '192'],
 ['2013', 'DANIEL', 'KINGS', 'M', '188'],
 ['2013', 'JACOB', 'QUEENS', 'M', '186'],
 ['2013', 'AIDEN', 'KINGS', 'M', '182'],
 ['2013', 'LEAH', 'KINGS', 'F', '179'],
 ['2013', 'NOAH', 'KINGS', 'M', '168'],
 ['2013', 'JOSEPH', 'KINGS', 'M', '166'],
 ['2013', 'MATTHEW', 'QUEENS', 'M', '161']]

In [21]:
# Ile razy dane imie wystąpiło ile razy?
(rows
 .map(lambda x: (x[1], int(x[4])))
 .groupByKey()
 .map(lambda x: {x[0]: sum(x[1])})
).collect()

[{'DAVID': 272},
 {'ETHAN': 412},
 {'JACOB': 388},
 {'ESTHER': 196},
 {'ISABELLA': 192},
 {'LEAH': 179},
 {'NOAH': 168},
 {'JOSEPH': 166},
 {'MATTHEW': 161},
 {'JAYDEN': 487},
 {'MOSHE': 219},
 {'SOPHIA': 204},
 {'DANIEL': 391},
 {'AIDEN': 182}]

In [26]:
x1 = sc.parallelize(['kot', 'pies', 'zajac', 'niedzwiedz'])
x2 = sc.parallelize(['wilk', 'pies', 'dzik', 'bocian'])

In [27]:
x1.reduce(lambda x,y: x + '-' + y)

'kot-pies-zajac-niedzwiedz'

In [28]:
def nazwa(x,y):
  return x + '-' + y

In [29]:
# łaczenie liczb znaków we wszystkich wyrazach x1
(x1
 .map(lambda x: [x, len(x)])
 .flatMap(lambda x: [x[1]])
).reduce(lambda x, y: x + y)

22

In [30]:
(x1
   .map(lambda x: (x, 'wartosc-wymagane-pole'))
).countByKey()

defaultdict(int, {'kot': 1, 'pies': 1, 'zajac': 1, 'niedzwiedz': 1})